# 7. Summary challenge


In this challenge, we're going to create the most unbreakable lock in the world - a quantum lock!

This lock is represented by a unitary operator *U*, which has all its eigenvalues equal to 1, except one. This exception, our "key" eigenstate, has an eigenvalue of -1.

$$U|\text{key}\rangle = -|\text{key}\rangle$$

The challenge, then, is to find a way of distinguishing the "key" eigenstate from the other eigenstates, given that the information is contained in the operator phase. Fortunately, there's a concept called **phase feedback** that can help.

When the correct eigenstate is entered, the -1 phase of U is transferred to the auxiliary qubit, changing its state from $|+\rangle$ to $|-\rangle$. The result of the measurement on the control qubit then indicates whether or not the correct eigenstate has been entered, indicating whether or not the latch is unlocked. In our case, $|1\rangle = H|-\rangle$
unlocks the lock, and $|0\rangle = H|+\rangle$ fails.

In this context, the key corresponds to a binary-encoded integer *m*, which represents our own "key" state

$$\begin{aligned}
U|n\rangle =
\begin{cases}
  -|n\rangle, & \text{if } n=m \\
  |n\rangle, & \text{if } n\neq m
\end{cases}
\end{aligned}$$

Once the lock has been constructed, we can test the different key eigenstates we've chosen and distinguish the one that unlocks our famous quantum lock.







Procedure:
* Choose the number of qubits according to the desired key size.
* Define the quantum machine.
* Define the quantum lock using the `FlipSign` operation.
* Use the `BasisState` operation to create the key.
* Define a function to open the lock.
* Define a function to check the key.

SOLUTION

In [1]:
#Challenge adapted from  https://pennylane.ai/qml/demos/tutorial_phase_kickback/

import pennylane as qml
from pennylane import numpy as np

# Define number of qubits
# For an n-digit combination, we need n+1 qubits (the first qubit is the control, and the others are the target)
num_wires = 5

dev = qml.device('default.qubit', wires = num_wires, shots = 1)


#Function that defines the quantum lock according to a secret key. Key length must be the number of qubit -1
def quantum_lock(secret_key):
  return qml.FlipSign(secret_key, wires = list(range(1, num_wires)))

def build_key(key):
  return qml.BasisState(key, wires = list(range(1, num_wires)))

In [2]:
@qml.qnode(dev)
def lock_mechanism(lock, key):
    build_key(key)
    qml.Hadamard(wires=0)  # Hadamard on auxiliary qubit
    qml.ctrl(lock, control=0)  #Controlled gate
    qml.Hadamard(wires=0)  # Hadamard on auxiliary qubit
    return qml.sample(wires=0)

In [3]:
def check_key(lock, key):
  if lock_mechanism(lock, key) ==1:
    print('BRAVO :  the lock is unlocked')
  else:
    print('FAIL: the lock is locked')

In [4]:
#Defining the secret key and defining the quantum lock
secret_key = np.array([1,1,1,0])
lock = quantum_lock(secret_key)

In [5]:
key_guess = np.array([0,0,0,0])
check_key(lock, key_guess)

FAIL: the lock is locked


In [6]:
key_guess = np.array([1,1,1,0])
check_key(lock, key_guess)

BRAVO :  the lock is unlocked


Challenge adapted from: https://pennylane.ai/qml/demos/tutorial_phase_kickback/